In [4]:
import re
rex = re.compile(r"\d+\n(\d{2}:\d{2}:\d{2}),\d{3} --> (\d{2}:\d{2}:\d{2}),\d{3}")

def chunk_srt_files(full_text, chunk_length):

    # split on the regex.
    splits = rex.split(full_text)[1:]

    # combine parts into a list of 3-tuples (start, end, txt)
    parts = []
    for i in range(0, len(splits), 3):
        start_time = splits[i]
        end_time = splits[i+1]
        content = splits[i+2].strip()
        parts.append((start_time, end_time, content))
        

    # combine multiple parts to get desired chunk length
    # will be a list of 3-tuples (start, end, txt)
    chunks = []
    ix = 0
    current_chunk_text = ""
    for i, part in enumerate(parts):
        current_chunk_text = current_chunk_text + " " + part[2]
        if len(current_chunk_text) > chunk_length or i == len(parts) - 1:
            # if we have a long enough chunk OR we are on the last piece of content...
            current_chunk = (
                parts[ix][0],  # starting timestamp
                part[1],
                current_chunk_text.strip()
            )
            chunks.append(current_chunk)
            ix = i  # we repeat this chunk one more time for overlap
            current_chunk_text =  part[2]

    return chunks

In [16]:
def save_to_srt(chunks, filename):
    with open(filename, 'w') as f:
        for idx, (start_time, end_time, content) in enumerate(chunks):
            f.write(f"{idx+1}\n")
            f.write(f"{start_time},000 --> {end_time},000\n")
            f.write(f"{content}\n\n")

In [5]:
with open("videos/transcripts_tiny/2.1.1 pandas intro.srt") as f:
    txt = f.read()

In [18]:
chunks = chunk_srt_files(txt, 500)

In [ ]:
save_to_srt(chunks, "videos/transcripts_chunked/2.1.1 pandas intro_chunked.srt")

In [14]:
import os 
os.getcwd()

'/home/jupyteach-msda/jupyteach-ai/videos'

In [13]:
os.chdir("/home/jupyteach-msda/jupyteach-ai/videos")


In [15]:
import os
import re

rex = re.compile(r"\d+\n(\d{2}:\d{2}:\d{2}),\d{3} --> (\d{2}:\d{2}:\d{2}),\d{3}")

#def chunk_srt_files(full_text, chunk_length):
    # ... (unchanged function from your original code)

def process_all_srt_files(input_folder_srt, output_folder_chunk, chunk_length):
    # Ensure the output folder exists, or create it if not
    if not os.path.exists(output_folder_chunk):
        os.makedirs(output_folder_chunk)

    # List all SRT files in the input folder
    srt_files = [f for f in os.listdir(input_folder_srt) if f.endswith('.srt')]

    for srt_file in srt_files:
        # Read the content of the SRT file
        with open(os.path.join(input_folder_srt, srt_file)) as f:
            txt = f.read()

        # Call the chunk_srt_files function to process the SRT content
        chunks = chunk_srt_files(txt, chunk_length)

        # Save the chunked content to the output folder
        output_file = os.path.join(output_folder_chunk, srt_file)
        with open(output_file, 'w') as f:
            for chunk in chunks:
                f.write(f"{chunk[0]} --> {chunk[1]}\n{chunk[2]}\n\n")

# Set the paths for the input and output folders and specify the chunk length
input_folder_srt = output_folder_srt
output_folder_chunk = "transcripts_tiny_output"
chunk_length = 1000

# Call the function to process all SRT files in the input folder
process_all_srt_files(input_folder_srt, output_folder_chunk, chunk_length)
